<a href="https://colab.research.google.com/github/SamiraHabib/Coursera_Capstone/blob/main/Coursera_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursera Capstone #

# Importando as bibliotecas #

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 81kB 5.0MB/s 
  Created wheel for folium: filename=folium-0.5.0-cp37-none-any.whl size=76240 sha256=63271da22701ffe195a78f5390135233317617aae82c2725ffd57ea879ba97d1
  Stored in directory: /root/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.5.0 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
Folium installed
Libraries imported.


# Definição das credenciais e version do Foursquare #

In [2]:
CLIENT_ID = '1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA' # your Foursquare ID
CLIENT_SECRET = 'ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA
CLIENT_SECRET:ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D


# Endereço do local escolhido #

In [3]:
#Exemplo - alterar
address = 'Rio de Janeiro, RJ'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-22.9110137 -43.2093727


# Localização #

In [4]:
#Localidade exemplo, mudar
latitude = -22.9110137
longitude = -43.2093727

# Define URL #

In [6]:
search_query = 'colégio'
radius = 2000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

colégio .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA&client_secret=ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D&ll=-22.9110137,-43.2093727&v=20180604&query=colégio&radius=2000&limit=50'

# Envia uma solicitação GET e examina os resultados #

In [7]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6082c28ed274d67370257c19'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/default_',
       'suffix': '.png'},
      'id': '52e81612bcbc57f1066b7a46',
      'name': 'Private School',
      'pluralName': 'Private Schools',
      'primary': True,
      'shortName': 'Private School'}],
    'hasPerk': False,
    'id': '4e60f0fb315114bf603f914f',
    'location': {'address': 'Tv. Dr. Araújo, 21',
     'cc': 'BR',
     'city': 'Rio de Janeiro',
     'country': 'Brasil',
     'distance': 691,
     'formattedAddress': ['Tv. Dr. Araújo, 21',
      'Rio de Janeiro, RJ',
      '20270-070',
      'Brasil'],
     'labeledLatLngs': [{'label': 'display',
       'lat': -22.914814836666505,
       'lng': -43.21470669286932}],
     'lat': -22.914814836666505,
     'lng': -43.21470669286932,
     'postalCode': '20270-070',
     'state': 'RJ'},
    'name': 'CARJ - Colégio Adventista do Rio de Janeiro',
   

## Pegando a parte relevante do JSON e transformando em um dataframe (pandas) ##

In [8]:
#transformando em dataframe
venues = results['response']['venues']
df_school = json_normalize(venues)
df_school.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4e60f0fb315114bf603f914f,CARJ - Colégio Adventista do Rio de Janeiro,"[{'id': '52e81612bcbc57f1066b7a46', 'name': 'P...",v-1619182222,False,"Tv. Dr. Araújo, 21",-22.914815,-43.214707,"[{'label': 'display', 'lat': -22.9148148366665...",691,20270-070,BR,Rio de Janeiro,RJ,Brasil,"[Tv. Dr. Araújo, 21, Rio de Janeiro, RJ, 20270...",NaN
1,4d30474405585941fce35b37,Colégio Qi,"[{'id': '52e81612bcbc57f1066b7a46', 'name': 'P...",v-1619182222,False,"R. Ibituruna, 37",-22.913802,-43.220259,"[{'label': 'display', 'lat': -22.9138020434751...",1158,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 37, Rio de Janeiro, RJ, 20271-0...",NaN
2,4c2b56d0b34ad13a8293e9ce,Colégio e Curso pH,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",v-1619182222,False,"R. Prof. Gabizo, 211",-22.917880,-43.221191,"[{'label': 'display', 'lat': -22.9178796014997...",1432,20271-064,BR,Rio de Janeiro,RJ,Brasil,"[R. Prof. Gabizo, 211, Rio de Janeiro, RJ, 202...",NaN
3,4c7cde823e90a1cda18743be,Pensi Colégio e Curso,"[{'id': '52e81612bcbc57f1066b7a46', 'name': 'P...",v-1619182222,False,"R. Ibituruna, 27",-22.913970,-43.219829,"[{'label': 'display', 'lat': -22.9139701021976...",1121,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 27 (Unidade Tijuca I), Rio de J...",Unidade Tijuca I
4,4dc1528422713750ba79c2ad,Colégio Militar do Rio de Janeiro (CMRJ),"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",v-1619182222,False,"R. S. Francisco Xavier, 267",-22.916797,-43.227150,"[{'label': 'display', 'lat': -22.9167972204570...",1933,20550-010,BR,Rio de Janeiro,RJ,Brasil,"[R. S. Francisco Xavier, 267, Rio de Janeiro, ...",NaN


# Define a informação de interesse e filtra o dataframe #

In [9]:
# Mantém somente as colunas que incluem venue name e qualquer coisa que é associada com a localização
filtered_columns = ['name', 'categories'] + [col for col in df_school.columns if col.startswith('location.')] + ['id']
df_school_filtered = df_school.loc[:, filtered_columns]

# função que extrai a categoria de venue 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filtra a categoria por cada linha do data frame
df_school_filtered['categories'] = df_school_filtered.apply(get_category_type, axis=1)

# limpa os nomes das colunas para manter somente o último termo
df_school_filtered.columns = [column.split('.')[-1] for column in df_school_filtered.columns]

df_school_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,CARJ - Colégio Adventista do Rio de Janeiro,Private School,"Tv. Dr. Araújo, 21",-22.914815,-43.214707,"[{'label': 'display', 'lat': -22.9148148366665...",691,20270-070,BR,Rio de Janeiro,RJ,Brasil,"[Tv. Dr. Araújo, 21, Rio de Janeiro, RJ, 20270...",NaN,4e60f0fb315114bf603f914f
1,Colégio Qi,Private School,"R. Ibituruna, 37",-22.913802,-43.220259,"[{'label': 'display', 'lat': -22.9138020434751...",1158,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 37, Rio de Janeiro, RJ, 20271-0...",NaN,4d30474405585941fce35b37
2,Colégio e Curso pH,High School,"R. Prof. Gabizo, 211",-22.917880,-43.221191,"[{'label': 'display', 'lat': -22.9178796014997...",1432,20271-064,BR,Rio de Janeiro,RJ,Brasil,"[R. Prof. Gabizo, 211, Rio de Janeiro, RJ, 202...",NaN,4c2b56d0b34ad13a8293e9ce
3,Pensi Colégio e Curso,Private School,"R. Ibituruna, 27",-22.913970,-43.219829,"[{'label': 'display', 'lat': -22.9139701021976...",1121,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 27 (Unidade Tijuca I), Rio de J...",Unidade Tijuca I,4c7cde823e90a1cda18743be
4,Colégio Militar do Rio de Janeiro (CMRJ),School,"R. S. Francisco Xavier, 267",-22.916797,-43.227150,"[{'label': 'display', 'lat': -22.9167972204570...",1933,20550-010,BR,Rio de Janeiro,RJ,Brasil,"[R. S. Francisco Xavier, 267, Rio de Janeiro, ...",NaN,4dc1528422713750ba79c2ad


# Avaliação dos dados: #

In [10]:
df_school_filtered.dtypes

name                 object
categories           object
address              object
lat                 float64
lng                 float64
labeledLatLngs       object
distance              int64
postalCode           object
cc                   object
city                 object
state                object
country              object
formattedAddress     object
crossStreet          object
id                   object
dtype: object

In [11]:
df_school_filtered.describe()

,lat,lng,distance
count,50.000000,50.000000,50.000000
mean,-22.913930,-43.217266,1565.660000
std,0.007523,0.010715,425.893855
min,-22.927834,-43.231521,691.000000
25%,-22.920066,-43.223853,1269.750000
50%,-22.914877,-43.220135,1674.500000
75%,-22.911467,-43.212006,1792.000000
max,-22.897638,-43.189540,2359.000000


In [12]:
df_school_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,CARJ - Colégio Adventista do Rio de Janeiro,Private School,"Tv. Dr. Araújo, 21",-22.914815,-43.214707,"[{'label': 'display', 'lat': -22.9148148366665...",691,20270-070,BR,Rio de Janeiro,RJ,Brasil,"[Tv. Dr. Araújo, 21, Rio de Janeiro, RJ, 20270...",NaN,4e60f0fb315114bf603f914f
1,Colégio Qi,Private School,"R. Ibituruna, 37",-22.913802,-43.220259,"[{'label': 'display', 'lat': -22.9138020434751...",1158,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 37, Rio de Janeiro, RJ, 20271-0...",NaN,4d30474405585941fce35b37
2,Colégio e Curso pH,High School,"R. Prof. Gabizo, 211",-22.917880,-43.221191,"[{'label': 'display', 'lat': -22.9178796014997...",1432,20271-064,BR,Rio de Janeiro,RJ,Brasil,"[R. Prof. Gabizo, 211, Rio de Janeiro, RJ, 202...",NaN,4c2b56d0b34ad13a8293e9ce
3,Pensi Colégio e Curso,Private School,"R. Ibituruna, 27",-22.913970,-43.219829,"[{'label': 'display', 'lat': -22.9139701021976...",1121,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 27 (Unidade Tijuca I), Rio de J...",Unidade Tijuca I,4c7cde823e90a1cda18743be
4,Colégio Militar do Rio de Janeiro (CMRJ),School,"R. S. Francisco Xavier, 267",-22.916797,-43.227150,"[{'label': 'display', 'lat': -22.9167972204570...",1933,20550-010,BR,Rio de Janeiro,RJ,Brasil,"[R. S. Francisco Xavier, 267, Rio de Janeiro, ...",NaN,4dc1528422713750ba79c2ad


# Número de nulos das colunas #

In [13]:
print("Number of NaN values for the column city :", df_school_filtered['city'].isnull().sum())
print("Number of NaN values for the column state :", df_school_filtered['state'].isnull().sum())
print("Number of NaN values for the column crossStreet :", df_school_filtered['crossStreet'].isnull().sum())

Number of NaN values for the column city : 14
Number of NaN values for the column state : 14
Number of NaN values for the column crossStreet : 44


In [14]:
df_school_filtered['state'].replace(np.nan,"RJ", inplace=True)

In [15]:
df_school_filtered['city'].replace(np.nan,"Rio de Janeiro", inplace=True)

In [16]:
print("Number of NaN values for the column state :", df_school_filtered['state'].isnull().sum())
print("Number of NaN values for the column city :", df_school_filtered['city'].isnull().sum())

Number of NaN values for the column state : 0
Number of NaN values for the column city : 0


In [17]:
df_school_filtered['categories'].value_counts()

School                             11
High School                         9
Private School                      6
Student Center                      3
College Academic Building           2
Community College                   2
College Classroom                   2
General College & University        2
College Arts Building               1
Music School                        1
University                          1
College Administrative Building     1
Library                             1
College Lab                         1
Middle School                       1
Church                              1
Courthouse                          1
Cafeteria                           1
Name: categories, dtype: int64

In [18]:
df_school_filtered.isnull().sum()

name                 0
categories           3
address             14
lat                  0
lng                  0
labeledLatLngs       0
distance             0
postalCode          32
cc                   0
city                 0
state                0
country              0
formattedAddress     0
crossStreet         44
id                   0
dtype: int64

# Dropando uma coluna #

In [29]:
df_school_filtered.drop('crossStreet', axis=1, inplace=True)

In [30]:
df_school_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,CARJ - Colégio Adventista do Rio de Janeiro,Private School,"Tv. Dr. Araújo, 21",-22.914815,-43.214707,"[{'label': 'display', 'lat': -22.9148148366665...",691,20270-070,BR,Rio de Janeiro,RJ,Brasil,"[Tv. Dr. Araújo, 21, Rio de Janeiro, RJ, 20270...",4e60f0fb315114bf603f914f
1,Colégio Qi,Private School,"R. Ibituruna, 37",-22.913802,-43.220259,"[{'label': 'display', 'lat': -22.9138020434751...",1158,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 37, Rio de Janeiro, RJ, 20271-0...",4d30474405585941fce35b37
2,Colégio e Curso pH,High School,"R. Prof. Gabizo, 211",-22.917880,-43.221191,"[{'label': 'display', 'lat': -22.9178796014997...",1432,20271-064,BR,Rio de Janeiro,RJ,Brasil,"[R. Prof. Gabizo, 211, Rio de Janeiro, RJ, 202...",4c2b56d0b34ad13a8293e9ce
3,Pensi Colégio e Curso,Private School,"R. Ibituruna, 27",-22.913970,-43.219829,"[{'label': 'display', 'lat': -22.9139701021976...",1121,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 27 (Unidade Tijuca I), Rio de J...",4c7cde823e90a1cda18743be
4,Colégio Militar do Rio de Janeiro (CMRJ),School,"R. S. Francisco Xavier, 267",-22.916797,-43.227150,"[{'label': 'display', 'lat': -22.9167972204570...",1933,20550-010,BR,Rio de Janeiro,RJ,Brasil,"[R. S. Francisco Xavier, 267, Rio de Janeiro, ...",4dc1528422713750ba79c2ad


In [31]:
df_school_filtered['name'].count()

50

In [32]:
print('There are {} uniques categories.'.format(len(df_school_filtered['categories'].unique())))

There are 19 uniques categories.


# Nome e categorias dos dados #

In [35]:
df_school_filtered.columns

Index(['name', 'categories', 'address', 'lat', 'lng', 'labeledLatLngs',
       'distance', 'postalCode', 'cc', 'city', 'state', 'country',
       'formattedAddress', 'id'],
      dtype='object')

In [36]:
df_school_filtered.name

0           CARJ - Colégio Adventista do Rio de Janeiro
1                                            Colégio Qi
2                                    Colégio e Curso pH
3                                 Pensi Colégio e Curso
4              Colégio Militar do Rio de Janeiro (CMRJ)
5                                      Colégio Pedro II
6                                  Colégio Ressurreição
7                                      Colégio Pedro II
8                           Pedrinho - Colégio Pedro II
9                                       Colégio Batista
10                                 Colégio Guanabarense
11                                    Colégio Curso SEI
12                             Colegio e curso superior
13                Colégio Miguel Couto - Unidade Tijuca
14                                     Colégio Cruzeiro
15                                colégio barbara Otoni
16                                 Colégio Maria Raythe
17                    Colégio Estadual Herbert D

In [37]:
df_school_filtered.categories

0                      Private School
1                      Private School
2                         High School
3                      Private School
4                              School
5                         High School
6                              School
7                              School
8                              School
9                                None
10       General College & University
11                             School
12                         University
13                     Student Center
14                        High School
15                     Student Center
16                             School
17              College Arts Building
18                        High School
19                     Private School
20                     Private School
21                  Community College
22    College Administrative Building
23          College Academic Building
24                        High School
25                        High School
26          

# Identificar as ids de escolas diferentes #

In [38]:
df_school_filtered[['id', 'name']]

,id,name
0,4e60f0fb315114bf603f914f,CARJ - Colégio Adventista do Rio de Janeiro
1,4d30474405585941fce35b37,Colégio Qi
2,4c2b56d0b34ad13a8293e9ce,Colégio e Curso pH
3,4c7cde823e90a1cda18743be,Pensi Colégio e Curso
4,4dc1528422713750ba79c2ad,Colégio Militar do Rio de Janeiro (CMRJ)
5,4ba39e43f964a520924c38e3,Colégio Pedro II
6,512b53c1e4b0fd3b1159b204,Colégio Ressurreição
7,5071a03ce4b0e1b65ea15489,Colégio Pedro II
8,4d873d9d8de9721e6b286951,Pedrinho - Colégio Pedro II
9,4dbfe92c4b2221ec2d640bac,Colégio Batista


In [40]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # gerar mapa centrado no entorno do RJ

# Circulo vermelho no centro
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='School',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# display map
venues_map

# Visualizar escolas no mapa ao redor da localização anteriormente citada #

In [39]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # gerar mapa centrado no entorno do RJ

# Circulo vermelho no centro
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='School',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# Escolas em azul ao entorno
for lat, lng, label in zip(df_school_filtered.lat, df_school_filtered.lng, df_school_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
       # popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map